In [3]:
import logging
import numpy as np
import ray
import time
from ray.exceptions import GetTimeoutError
import cProfile

if ray.is_initialized:
    ray.shutdown()
ray.init()

2023-04-16 15:37:04,669	INFO worker.py:1544 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


Python version:,3.10.6
Ray version:,2.3.1
Dashboard:,http://127.0.0.1:8265


### Remote Objects example

In [12]:
num_list = [23, 42, 93]

obj_ref = ray.put(num_list)
print(obj_ref)

val = ray.get(obj_ref)
print(val)

results = ray.get([ray.put(i) for i in range(10)])
print(results)

ObjectRef(00ffffffffffffffffffffffffffffffffffffff0100000014000000)
[23, 42, 93]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


### Passing Objects by Reference


In [ ]:
@ray.remote
def echo(x):
    print(f"current value of argument: {x}")

In [8]:
x = list(range(10))
obj_ref_x = ray.put(x)
y = 25

echo.remote(y)
echo.remote(obj_ref_x)

ObjectRef(d695f922effe6d99ffffffffffffffffffffffff0100000001000000)

(echo pid=60637) current value of argument x: 25
(echo pid=60628) current value of argument x: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


In [9]:

x = list(range(20))
obj_ref_x = ray.put(x)
echo.remote({"obj": obj_ref_x})
echo.remote([obj_ref_x])

ObjectRef(71b133a11e1c461cffffffffffffffffffffffff0100000001000000)

(echo pid=60637) current value of argument x: [ObjectRef(00ffffffffffffffffffffffffffffffffffffff0100000013000000)]
(echo pid=60628) current value of argument x: {'obj': ObjectRef(00ffffffffffffffffffffffffffffffffffffff0100000013000000)}


### Handling Long-Running Tasks with Timeout

In [10]:
@ray.remote
def long_running_function():
    time.sleep(10)
    return 42

def time_out_funtion():
    obj_ref = long_running_function.remote()
    try:
        ray.get(obj_ref, timeout=6)
    except GetTimeoutError:
        print("`get` timed out")

print("start task")
cProfile.run("time_out_funtion()")

start task
`get` timed out
         740 function calls (737 primitive calls) in 6.006 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    6.005    6.005 287432207.py:6(time_out_funtion)
        4    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:1053(_handle_fromlist)
       14    0.000    0.000    0.000    0.000 <string>:1(<lambda>)
        1    0.000    0.000    6.006    6.006 <string>:1(<module>)
        2    0.000    0.000    0.000    0.000 __init__.py:1455(debug)
        2    0.000    0.000    0.000    0.000 __init__.py:1724(isEnabledFor)
        2    0.000    0.000    0.000    0.000 __init__.py:980(__getitem__)
        2    0.000    0.000    0.002    0.001 _channel.py:1021(__call__)
        2    0.000    0.000    0.000    0.000 _channel.py:118(__init__)
        2    0.000    0.000    0.000    0.000 _channel.py:1350(__new__)
        2    0.000    0.000    0.000    0.000 _chann

---

In [13]:
ray.shutdown()